# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


!pip install gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,narsaq,60.92,-46.05,33.80,43,100,19.46,GL,1579585175
1,1,mataura,-46.19,168.86,67.03,71,100,7.78,NZ,1579585160
2,2,punta arenas,-53.15,-70.92,55.40,66,75,17.22,CL,1579585177
3,3,bairiki,1.33,172.98,89.60,66,40,4.70,KI,1579585180
4,4,derzhavinsk,51.10,66.32,15.87,94,100,16.35,KZ,1579585180


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key = g_key)


In [11]:
#Heatmap of humidity

locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights = humidity, dissipating=False, max_intensity=300,point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#NArrowing down cities that fit criteria and drop results with no values

narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"]< 80)& (city_data_df["Max Temp"]>70)\
                                   & (city_data_df["Wind Speed"]<10)\
                                   &(city_data_df["Cloudiness"]== 0)].dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,cape town,-33.93,18.42,71.01,82,0,3.36,ZA,1579584964
219,219,doha,25.29,51.53,75.00,63,0,6.93,QA,1579585266
334,334,sur,22.57,59.53,72.23,47,0,1.50,OM,1579585314
410,410,san patricio,19.22,-104.70,79.11,72,0,5.41,MX,1579585362
459,459,port lincoln,-34.73,135.87,72.61,50,0,9.15,AU,1579585390


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df= narrowed_city_df[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
30,cape town,ZA,-33.93,18.42,
219,doha,QA,25.29,51.53,
334,sur,OM,22.57,59.53,
410,san patricio,MX,19.22,-104.70,
459,port lincoln,AU,-34.73,135.87,


In [15]:
params = {
    "radius":5000,
    "type":"lodging",
    "key":g_key
}

for index, row in hotel_df.iterrows():
    #get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"]=f"{lat},{lng}"
    
    #use the search term: hotel and out lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]= name_address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result...skipping")
hotel_df


,City,Country,Lat,Lng,Hotel Name
30,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town
219,doha,QA,25.29,51.53,Mövenpick Hotel Doha
334,sur,OM,22.57,59.53,Sur Plaza Hotel
410,san patricio,MX,19.22,-104.70,Hotel Cabo Blanco
459,port lincoln,AU,-34.73,135.87,Port Lincoln Hotel


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))